In [1]:
#Python
#import module for predict
import os
import numpy as np
import configparser
import cv2
import sys
from matplotlib import pyplot as plt
#Keras
sys.path.insert(0, './lib/')
# help_functions.py
from help_functions import *
# extract_patches.py
from extract_patches import *
# pre_processing.py
from pre_processing import *



In [16]:
path_data = './conj_data/'
def width_padding(img, pad_size):
    h,w = np.shape(img)
    #print('pad size', pad_size)
    
    if pad_size % 2 ==1: #odd number
        pad01 = np.zeros((h, pad_size//2))
        pad02 = np.zeros((h, pad_size//2 +1))
    
        result = np.hstack([pad01, img])
        result = np.hstack([result, pad02])
        
    else:
        pad01 = np.zeros((h, pad_size//2))
    
        result = np.hstack([pad01, img])
        result = np.hstack([result, pad01])
        
    print(np.shape(img), ' ',np.shape(result))
    return result
    
def height_padding(img,pad_size):
    h,w = np.shape(img)
    #print('pad size : ',pad_size)
    
    if pad_size % 2 ==1: #odd number
        pad01 = np.zeros((pad_size//2,w ))
        pad02 = np.zeros((pad_size//2 +1 , w))
        
        result = np.vstack([pad01, img])
        result = np.vstack([result,pad02])
        
    else:
        pad01 = np.zeros((pad_size//2, w))
    
        result = np.vstack([pad01, img])
        result = np.vstack([result, pad01])
    print(np.shape(img), ' ',np.shape(result))
    return result

    patches_imgs_test = extract_ordered(test_imgs,patch_h,patch_w)
    #data_consistency_check(test_imgs, test_grds)

    print ("\n[get_data_testing_fucn] test PATCHES images/grds shape:")
    print (patches_imgs_test.shape)
    print ("[get_data_testing_fucn] test PATCHES images range (min-max): {} - {}".format(str(np.min(patches_imgs_test)), str(np.max(patches_imgs_test))))

    return patches_imgs_test

def width_avg_padding(img, pad_size):
    h,w = np.shape(img)
    #print('pad size', pad_size)
    avg_val = np.mean(img)
    
    if pad_size % 2 ==1: #odd number
        pad01 = np.full((h, pad_size//2),avg_val)
        pad02 = np.full((h, pad_size//2 +1),avg_val)
    
        result = np.hstack([pad01, img])
        result = np.hstack([result, pad02])
        
    else:
        pad01 = np.full((h, pad_size//2),avg_val)
    
        result = np.hstack([pad01, img])
        result = np.hstack([result, pad01])
        
    print(np.shape(img), ' ',np.shape(result))
    return result
    
def height_avg_padding(img,pad_size):
    h,w = np.shape(img)
    #print('pad size : ',pad_size)
    avg_val = np.mean(img)
    if pad_size % 2 ==1: #odd number
        pad01 = np.full((pad_size//2,w ),avg_val)
        pad02 = np.full((pad_size//2 +1 , w),avg_val)
        
        result = np.vstack([pad01, img])
        result = np.vstack([result,pad02])
        
    else:
        pad01 = np.full((pad_size//2, w),avg_val)
    
        result = np.vstack([pad01, img])
        result = np.vstack([result, pad01])
    print(np.shape(img), ' ',np.shape(result))
    return result

    patches_imgs_test = extract_ordered(test_imgs,patch_h,patch_w)
    #data_consistency_check(test_imgs, test_grds)

    print ("\n[get_data_testing_fucn] test PATCHES images/grds shape:")
    print (patches_imgs_test.shape)
    print ("[get_data_testing_fucn] test PATCHES images range (min-max): {} - {}".format(str(np.min(patches_imgs_test)), str(np.max(patches_imgs_test))))

    return patches_imgs_test

In [15]:
pad_size = 2
h= [[8,4,2],[3,15,11],[9,3,1]]
h = np.asarray(h, dtype='uint8')
print(np.shape(h))
pad01 = np.zeros((int(pad_size//2),3))
result = np.vstack([pad01, h])


print(result)

avg_val = int(np.mean(h))
pad01 = np.full((int(pad_size//2),3),avg_val)
result = np.vstack([pad01, h])

print(result)

(3, 3)
[[ 0.  0.  0.]
 [ 8.  4.  2.]
 [ 3. 15. 11.]
 [ 9.  3.  1.]]
[[ 6  6  6]
 [ 8  4  2]
 [ 3 15 11]
 [ 9  3  1]]


In [3]:
# 1. image size 조절 (DRIVE와 같은 크기로 하자)
# 2. image 크기가 작은 경우, 평균값으로 배경을 채운다.
# 3. image 크기가 큰 경우, 여러 조각으로 나누어 데이터 수를 증가시킨다.

# DRIVE image information
# img_height = 584
# img_width = 565

# 사이즈만 맞춰서 특정 폴더에 넣어버리

    
def conj_img_size_preprocessing(data_path):
    DRIVE_height = 584
    DRIVE_width = 565
    
    
    cnt = 1
    
    for path, subdirs, files in os.walk(data_path): #list all files, directories in the path
        for i in range(len(files)):
            tempImg = cv2.imread(data_path + files[i])
  
            if (len(np.shape(tempImg)) >2 ):
                tempImg =  tempImg[:,:,1]
            
            h, w = tempImg.shape
           
            if (w < DRIVE_width):
                pad_size = DRIVE_width - w

                tempImg = width_padding(tempImg, pad_size)
                
            elif (w > DRIVE_width):
                # 대충 crop하자
                tempImg = tempImg[:, : DRIVE_width]      
            else:
                pass
                #print("conj width == DRIVE_width")

                
            if (h < DRIVE_height):
                pad_size = DRIVE_height - h
                tempImg = height_padding(tempImg, pad_size)
                
            elif (h > DRIVE_height):
                tempImg = tempImg[: DRIVE_height, :]
                
            else:
                pass
                #print("conj height ==  DRIVE_height")
            
                
            if os.path.isdir('./conj_train') == False:
                os.mkdir('./conj_train')
            else:
                print('already exist the folder in this path : {}'.format('./conj_train'))
            
            #print('[WHAT THE FUCK]', np.shape(tempImg))
            cv2.imwrite('./conj_train/'+'/train_data_'+str(cnt)+'.png', tempImg)
            cnt = cnt+1
            
    

conj_img_size_preprocessing(path_data)


(569, 565)   (584, 565)
already exist the folder in this path : ./conj_train
[WHAT THE FUCK] (584, 565)
already exist the folder in this path : ./conj_train
[WHAT THE FUCK] (584, 565)
(423, 565)   (584, 565)
already exist the folder in this path : ./conj_train
[WHAT THE FUCK] (584, 565)
already exist the folder in this path : ./conj_train
[WHAT THE FUCK] (584, 565)
(536, 565)   (584, 565)
already exist the folder in this path : ./conj_train
[WHAT THE FUCK] (584, 565)
(530, 565)   (584, 565)
already exist the folder in this path : ./conj_train
[WHAT THE FUCK] (584, 565)
(342, 565)   (584, 565)
already exist the folder in this path : ./conj_train
[WHAT THE FUCK] (584, 565)
(558, 546)   (558, 565)
(558, 565)   (584, 565)
already exist the folder in this path : ./conj_train
[WHAT THE FUCK] (584, 565)
(497, 565)   (584, 565)
already exist the folder in this path : ./conj_train
[WHAT THE FUCK] (584, 565)
(480, 565)   (584, 565)
already exist the folder in this path : ./conj_train
[WHAT THE F

In [17]:
# 1. image size 조절 (DRIVE와 같은 크기로 하자)
# 2. image 크기가 작은 경우, 평균값으로 배경을 채운다.
# 3. image 크기가 큰 경우, 여러 조각으로 나누어 데이터 수를 증가시킨다.

# DRIVE image information
# img_height = 584
# img_width = 565

# 사이즈만 맞춰서 특정 폴더에 넣어버리

    
def conj_img_size_avg_preprocessing(data_path):
    DRIVE_height = 584
    DRIVE_width = 565
    
    
    cnt = 1
    
    for path, subdirs, files in os.walk(data_path): #list all files, directories in the path
        for i in range(len(files)):
            tempImg = cv2.imread(data_path + files[i])
  
            if (len(np.shape(tempImg)) >2 ):
                tempImg =  tempImg[:,:,1]
            
            h, w = tempImg.shape
           
            if (w < DRIVE_width):
                pad_size = DRIVE_width - w

                tempImg = width_avg_padding(tempImg, pad_size)
                
            elif (w > DRIVE_width):
                # 대충 crop하자
                tempImg = tempImg[:, : DRIVE_width]      
            else:
                pass
                #print("conj width == DRIVE_width")

                
            if (h < DRIVE_height):
                pad_size = DRIVE_height - h
                tempImg = height_avg_padding(tempImg, pad_size)
                
            elif (h > DRIVE_height):
                tempImg = tempImg[: DRIVE_height, :]
                
            else:
                pass
                #print("conj height ==  DRIVE_height")
            
                
            if os.path.isdir('./conj_train_avg') == False:
                os.mkdir('./conj_train_avg')
            else:
                print('already exist the folder in this path : {}'.format('./conj_train_avg'))
            
            #print('[WHAT THE FUCK]', np.shape(tempImg))
            cv2.imwrite('./conj_train_avg/'+'/train_data_'+str(cnt)+'.png', tempImg)
            cnt = cnt+1
            
    

conj_img_size_avg_preprocessing(path_data)


(569, 565)   (584, 565)
already exist the folder in this path : ./conj_train_avg
(423, 565)   (584, 565)
already exist the folder in this path : ./conj_train_avg
already exist the folder in this path : ./conj_train_avg
(536, 565)   (584, 565)
already exist the folder in this path : ./conj_train_avg
(530, 565)   (584, 565)
already exist the folder in this path : ./conj_train_avg
(342, 565)   (584, 565)
already exist the folder in this path : ./conj_train_avg
(558, 546)   (558, 565)
(558, 565)   (584, 565)
already exist the folder in this path : ./conj_train_avg
(497, 565)   (584, 565)
already exist the folder in this path : ./conj_train_avg
(480, 565)   (584, 565)
already exist the folder in this path : ./conj_train_avg
(388, 565)   (584, 565)
already exist the folder in this path : ./conj_train_avg
(380, 565)   (584, 565)
already exist the folder in this path : ./conj_train_avg
(459, 565)   (584, 565)
already exist the folder in this path : ./conj_train_avg
(571, 540)   (571, 565)
(571

In [4]:
HEIGHT_IMG = 584
WIDTH_IMG = 565
NUM_IMG = 0
CONJ_IMG_LIST = []

print(path_data)

for path, subdirs, files in os.walk('./conj_train/'): #list all files, directories in the path
        for i in range(len(files)):
            tempImg = cv2.imread('./conj_train/' + files[i], 0)
            print(np.shape(tempImg))
            CONJ_IMG_LIST.append(tempImg)

./conj_data/
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)
(584, 565)


In [ ]:
# make mask
